In [11]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import mne, re, os

from utils import *

In [12]:
DATA_PATH_TUH = '/home/williamtheodor/Documents/DL for EEG Classification/113/aaaaaqtx/'
FILE = DATA_PATH_TUH + 's002_2014_02_08/01_tcp_ar/aaaaaqtx_s002_t000.edf'

#PARCELLATION = 'aparc.a2009s'
PARCELLATION = 'HCPMMP1_combined'

In [13]:
def fix_average_naming(raw):
    # is the average naming a problem?
    reSTR = r"(?<=EEG )(.*)(?=-)"
    reLowC = ['FP1', 'FP2', 'FZ', 'CZ', 'PZ']

    for channel_name in raw.ch_names:
        if re.search(reSTR, channel_name) and re.search(reSTR, channel_name).group() in reLowC:
            lowC = channel_name[0:5]+channel_name[5].lower()+channel_name[6:]
            mne.channels.rename_channels(raw.info, {channel_name: re.findall(reSTR, lowC)[0]})
        elif channel_name == "PHOTIC-REF":
            mne.channels.rename_channels(raw.info, {channel_name: "PHOTIC"})
        elif re.search(reSTR, channel_name):
            mne.channels.rename_channels(raw.info, {channel_name: re.findall(reSTR, channel_name)[0]})

    return raw

def pick_and_rename_TUH_channels(raw):

    raw = fix_average_naming(raw)

    mne.channels.rename_channels(raw.info, {'Fp1': 'FP1', 'Fp2': 'FP2', 'Fz': 'FZ', 'Cz': 'CZ', 'Pz': 'PZ'})
    mne.channels.rename_channels(raw.info, {'T3': 'T7', 'T4': 'T8'})

    EEG_20_div = [
                'FP1', 'FP2',
        'F7', 'F3', 'FZ', 'F4', 'F8',
        'T7', 'C3', 'CZ', 'C4', 'T8',
        'T5', 'P3', 'PZ', 'P4', 'T6',
                 'O1', 'O2'
    ]
    
    raw.pick_channels(ch_names=EEG_20_div)
    raw.reorder_channels(EEG_20_div)

    return raw

In [21]:
def get_TUH_raw(FILE, high_pass=0.5, low_pass=70):
    raw = mne.io.read_raw_edf(FILE, verbose=False, preload=True)
    raw = pick_and_rename_TUH_channels(raw)

    mne.datasets.eegbci.standardize(raw)  # Set channel names
    montage = mne.channels.make_standard_montage('standard_1020')
    raw = raw.set_eeg_reference(ref_channels='average', projection=True, verbose=False)
    raw = raw.set_montage(montage)

    raw.filter(high_pass, low_pass, verbose=False)

    return raw

def get_TUH_annotation(raw, WINDOW_LENGTH=4, SAMPLNIG_FREQ=256):
    new_onset = []
    new_duration = []
    new_description = []

    number_windows = int(raw.get_data().shape[1] / SAMPLNIG_FREQ / WINDOW_LENGTH)

    for window_idx in range(number_windows):
        new_onset.append(window_idx * WINDOW_LENGTH)
        new_duration.append(WINDOW_LENGTH)
        new_description.append(f'chop {window_idx}')

    new_onset = np.array(new_onset, dtype=np.float64)
    new_duration = np.array(new_duration, dtype=np.float64)

    annotations = mne.Annotations(onset=new_onset, duration=new_duration, description=new_description)

    return annotations

In [22]:
raw_full = get_TUH_raw(FILE)
annotations = get_TUH_annotation(raw_full)
window_dict = get_window_dict(raw_full, annotations)

In [25]:
raw = window_dict['chop 0'][0]
fwd = get_fwd(raw.info, trans, src_path, bem_path)
cov = get_cov(raw)
compute_inverse = make_fast_inverse_operator(raw.info, fwd, cov, snr=SNR)

In [47]:
SNR = 100
subjects_dir, subject, trans, src_path, bem_path = get_fsaverage()
src = get_src(src_path)

labels = get_labels(subjects_dir, parcellation_name = 'HCPMMP1_combined')
label_names = [label.name for label in np.array(labels).flatten()]

bands = {'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
        'Gamma': (30, 70)}

activity = np.zeros((len(window_dict.keys()), len(list(bands.keys())), len(label_names)))

for i, window_key in tqdm(enumerate(window_dict.keys())):
    for j, band_key in enumerate(bands.keys()):
        raw_temp = window_dict[window_key][0]
        raw_temp.filter(bands[band_key][0], bands[band_key][1], verbose=False)

        stc = compute_inverse(raw_temp)
        activity[i, j] = np.concatenate(get_power_per_label(stc, labels, standardize=False)).reshape(1, -1)

0it [00:00, ?it/s]/tmp/ipykernel_8731/748505717.py:19: RuntimeWarning: filter_length (1691) is longer than the signal (1025), distortion is likely. Reduce filter length or filter a longer signal.
  raw_temp.filter(bands[band_key][0], bands[band_key][1], verbose=False)
1it [00:02,  2.67s/it]/tmp/ipykernel_8731/748505717.py:19: RuntimeWarning: filter_length (1691) is longer than the signal (1025), distortion is likely. Reduce filter length or filter a longer signal.
  raw_temp.filter(bands[band_key][0], bands[band_key][1], verbose=False)
2it [00:05,  2.64s/it]/tmp/ipykernel_8731/748505717.py:19: RuntimeWarning: filter_length (1691) is longer than the signal (1025), distortion is likely. Reduce filter length or filter a longer signal.
  raw_temp.filter(bands[band_key][0], bands[band_key][1], verbose=False)
3it [00:07,  2.60s/it]/tmp/ipykernel_8731/748505717.py:19: RuntimeWarning: filter_length (1691) is longer than the signal (1025), distortion is likely. Reduce filter length or filter a 

In [49]:
activity

array([[[5.55347057e-73, 1.52342732e-72, 9.35397647e-73, ...,
         5.56181419e-74, 1.21536655e-73, 1.37431820e-73],
        [5.51666068e-73, 1.50933081e-72, 9.19502765e-73, ...,
         5.45829253e-74, 1.18818167e-73, 1.34821488e-73],
        [2.63586921e-78, 7.34008622e-78, 4.21115966e-78, ...,
         2.64330451e-79, 5.61790270e-79, 6.15081344e-79],
        [3.27589509e-84, 9.08818614e-84, 5.10293800e-84, ...,
         3.31679576e-85, 7.25601413e-85, 7.67472439e-85],
        [1.49019304e-89, 4.09389361e-89, 2.29775555e-89, ...,
         1.50109333e-90, 3.35611958e-90, 3.52116873e-90]],

       [[2.45244244e-72, 6.70544647e-72, 5.95317686e-72, ...,
         2.21797241e-73, 7.13118754e-73, 8.71339942e-73],
        [1.88867534e-72, 5.12375251e-72, 4.86454470e-72, ...,
         1.27545185e-73, 5.71569308e-73, 6.86526437e-73],
        [9.94586221e-78, 2.68488812e-77, 2.50723528e-77, ...,
         6.89127648e-79, 2.98761455e-78, 3.59725457e-78],
        [1.59964441e-83, 4.36780039e-8

In [41]:
np.array([1,2]).reshape(1,-1).shape

(1, 2)